In [2]:
%matplotlib inline
import numpy as np

In [8]:


def plot_sim(data1 data2):
    """
    data1: free Zinc
    data2: chelatez Zinc
    """
    args = data['args']

    AE = []
    for i in range(len(args['bg_levels'])):
        AE.append([[1,4]])
        AE.append([[1,2]])
        AE.append([[1,1]])
    
    fig, AX = ge.figure(axes_extents=AE, figsize=(3.5,.25), wspace=0., left=.2)
    
    for ibg, bg in enumerate(args['bg_levels']):

        t0 = ibg*len(args['NSTIMs'])*len(args['stimSEEDS'])*len(args['bgSEEDS'])*args['duration_per_bg_level']
        t1 = (ibg+1)*len(args['NSTIMs'])*len(args['stimSEEDS'])*len(args['bgSEEDS'])*args['duration_per_bg_level']

        tcond = (data['t']>=t0) & (data['t']<t1)
        
        AX[3*ibg].plot(data2['t'][tcond], data2['Vm_soma'][tcond], color=ge.green, label='chelated-Zinc', lw=1.5)
        AX[3*ibg].plot(data['t'][tcond], data['Vm_soma'][tcond], color='k', label='free-Zinc', lw=1.5)
        AX[3*ibg].plot([t0,t1], [-75,-75], 'k--', lw=0.5)
        
        for i, sp0 in enumerate(data['BG_raster']):
            sp = np.array(sp0)
            cond = (sp>=t0) & (sp<t1)
            AX[3*ibg+1].scatter(sp[cond], i*np.ones(len(sp[cond])), color=ge.purple, s=2)
        for i, sp0 in enumerate(data['STIM_raster']):
            sp = np.array(sp0)
            cond = (sp>=t0) & (sp<t1)
            AX[3*ibg+1].scatter(sp[cond], i*np.ones(len(sp[cond])), color=ge.orange, s=4)
            
        ge.annotate(AX[3*ibg+1], '$\\nu_{bg}$=%.1fHz' % bg, (0,0), color=ge.purple, rotation=90, ha='right')
        ge.set_plot(AX[3*ibg], [], xlim=[t0, t1])#, ylabel='Vm (mV)')
        ge.draw_bar_scales(AX[3*ibg], Ybar_label_format="%.1fmV", Xbar_label_format="%.0fms",
                           Ybar_fraction=.25, Xbar_fraction=.05, loc=(0.005,.99), orientation='right-bottom')
        ge.annotate(AX[3*ibg], ' -75mV', (t1,-75), xycoords='data', va='center')
        ge.set_plot(AX[3*ibg+1], [], xlim=[t0, t1])#, ylabel='synapse ID')
        ge.set_plot(AX[3*ibg+2], [], xlim=[t0, t1])#, ylabel='synapse ID')
        # AX[3*ibg+2].axis('off')

    y1 = AX[1].get_ylim()[1]
    for istim, nstim in enumerate(args['NSTIMs']):
        ge.annotate(AX[1], ' %i syn.' % nstim,
                    (args['stim_delay']+istim*args['duration_per_bg_level']*len(args['stimSEEDS']),y1/2),
                    xycoords='data', color=ge.orange, va='center', ha='right', rotation=90)
        
